# Chapter 7. 텍스트 문서의 범주화 - (4) IMDB 전체 데이터로 전이학습

- 앞선 전이학습 실습과는 달리, IMDB 영화리뷰 데이터셋 전체를 사용하며 문장 수는 10개 -> 20개로 조정한다
- IMDB 영화 리뷰 데이터를 다운로드 받아 data 디렉토리에 압축 해제한다
    - 다운로드 : http://ai.stanford.edu/~amaas/data/sentiment/
    - 저장경로 : data/aclImdb

In [1]:
import os
import config
from dataloader.loader import Loader
from preprocessing.utils import Preprocess, remove_empty_docs
from dataloader.embeddings import GloVe
from model.cnn_document_model import DocumentModel, TrainingParameters
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np

Using TensorFlow backend.


## 학습 파라미터 설정

In [2]:
# 학습된 모델을 저장할 디렉토리 생성
if not os.path.exists(os.path.join(config.MODEL_DIR, 'imdb')):
    os.makedirs(os.path.join(config.MODEL_DIR, 'imdb'))

# 학습 파라미터 설정
train_params = TrainingParameters('imdb_transfer_tanh_activation', 
                                  model_file_path = config.MODEL_DIR+ '/imdb/full_model_10.hdf5',
                                  model_hyper_parameters = config.MODEL_DIR+ '/imdb/full_model_10.json',
                                  model_train_parameters = config.MODEL_DIR+ '/imdb/full_model_10_meta.json',
                                  num_epochs=30,
                                  batch_size=128)

## IMDB 데이터셋 로드

In [3]:
# 다운받은 IMDB 데이터 로드: 학습셋 전체 사용
train_df = Loader.load_imdb_data(directory = 'train')
# train_df = train_df.sample(frac=0.05, random_state = train_params.seed)
print(f'train_df.shape : {train_df.shape}')

test_df = Loader.load_imdb_data(directory = 'test')
print(f'test_df.shape : {test_df.shape}')

# 텍스트 데이터, 레이블 추출
corpus = train_df['review'].tolist()
target = train_df['sentiment'].tolist()
corpus, target = remove_empty_docs(corpus, target)
print(f'corpus size : {len(corpus)}')
print(f'target size : {len(target)}')

train_df.shape : (25000, 2)
test_df.shape : (25000, 2)
corpus size : 25000
target size : 25000


## 인덱스 시퀀스 생성

In [4]:
# 앞선 전이학습 실습과 달리, 문장 개수를 10개 -> 20개로 상향
Preprocess.NUM_SENTENCES = 20

# 학습셋을 인덱스 시퀀스로 변환
preprocessor = Preprocess(corpus=corpus)
corpus_to_seq = preprocessor.fit()

Found 28654 unique tokens.
All documents processed.ocessed.

In [5]:
print(f'corpus_to_seq size : {len(corpus_to_seq)}')
print(f'corpus_to_seq[0] size : {len(corpus_to_seq[0])}')

corpus_to_seq size : 25000
corpus_to_seq[0] size : 600


In [6]:
# 테스트셋을 인덱스 시퀀스로 변환
test_corpus = test_df['review'].tolist()
test_target = test_df['sentiment'].tolist()
test_corpus, test_target = remove_empty_docs(test_corpus, test_target)
test_corpus_to_seq = preprocessor.transform(test_corpus)

All documents processed.ocessed.

In [7]:
print(f'test_corpus_to_seq size : {len(test_corpus_to_seq)}')
print(f'test_corpus_to_seq[0] size : {len(test_corpus_to_seq[0])}')

test_corpus_to_seq size : 25000
test_corpus_to_seq[0] size : 600


In [8]:
# 학습셋, 테스트셋 준비
x_train = np.array(corpus_to_seq)
x_test = np.array(test_corpus_to_seq)
y_train = np.array(target)
y_test = np.array(test_target)

print(f'x_train.shape : {x_train.shape}')
print(f'y_train.shape : {y_train.shape}')
print(f'x_test.shape : {x_test.shape}')
print(f'y_test.shape : {y_test.shape}')

x_train.shape : (25000, 600)
y_train.shape : (25000,)
x_test.shape : (25000, 600)
y_test.shape : (25000,)


## GloVe 임베딩 초기화

In [9]:
# GloVe 임베딩 초기화 - glove.6B.50d.txt pretrained 벡터 사용
glove = GloVe(50)
initial_embeddings = glove.get_embedding(preprocessor.word_index)
print(f'initial_embeddings.shape : {initial_embeddings.shape}')

Reading 50 dim GloVe vectors
Found 400000 word vectors.
words not found in embeddings: 499
initial_embeddings.shape : (28656, 50)


##  훈련된 모델 로드

- HandsOn03에서 아마존 리뷰 데이터로 학습한 CNN 모델을 로드한다.
- DocumentModel 클래스의 load_model로 모델을 로드하고, load_model_weights로 학습된 가중치를 가져온다. 
- 그 후, GloVe.update_embeddings 함수로 GloVe 초기화 임베딩을 업데이트한다

In [10]:
# 모델 하이퍼파라미터 로드
model_json_path = os.path.join(config.MODEL_DIR, 'amazonreviews/model_06.json')
amazon_review_model = DocumentModel.load_model(model_json_path)

# 모델 가중치 로드
model_hdf5_path = os.path.join(config.MODEL_DIR, 'amazonreviews/model_06.hdf5')
amazon_review_model.load_model_weights(model_hdf5_path)


Vocab Size = 43197  and the index of vocabulary words passed has 43195 words
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
# 모델 임베딩 레이어 추출
learned_embeddings = amazon_review_model.get_classification_model().get_layer('imdb_embedding').get_weights()[0]
print(f'learned_embeddings size : {len(learned_embeddings)}')

# 기존 GloVe 모델을 학습된 임베딩 행렬로 업데이트한다
glove.update_embeddings(preprocessor.word_index, 
                        np.array(learned_embeddings), 
                        amazon_review_model.word_index)

# 업데이트된 임베딩을 얻는다
initial_embeddings = glove.get_embedding(preprocessor.word_index)


learned_embeddings size : 43197
23629 words are updated out of 28654


##  IMDB 전이학습 모델 생성

In [12]:
# 분류 모델 생성 : IMDB 리뷰 데이터를 입력받아 이진분류를 수행하는 모델 생성
imdb_model = DocumentModel(vocab_size=preprocessor.get_vocab_size(),
                           word_index = preprocessor.word_index,
                           num_sentences=Preprocess.NUM_SENTENCES,     
                           embedding_weights=initial_embeddings,
                           embedding_regularizer_l2 = 0.0,
                           conv_activation = 'tanh',
                           train_embedding = True,   # 임베딩 레이어의 가중치 학습함
                           learn_word_conv = False,  # 단어 수준 conv 레이어의 가중치 학습 안 함
                           learn_sent_conv = False,  # 문장 수준 conv 레이어의 가중치 학습 안 함
                           hidden_dims=64,                                        
                           input_dropout=0.1, 
                           hidden_layer_kernel_regularizer=0.01,
                           final_layer_kernel_regularizer=0.01)

# 가중치 업데이트 : 생성한 imdb_model 모델에서 다음의 각 레이어들의 가중치를 위에서 로드한 가중치로 갱신한다
for l_name in ['word_conv','sentence_conv','hidden_0', 'final']:
    new_weights = amazon_review_model.get_classification_model().get_layer(l_name).get_weights()
    imdb_model.get_classification_model().get_layer(l_name).set_weights(weights=new_weights)

Vocab Size = 28656  and the index of vocabulary words passed has 28654 words


## 모델 학습 및 평가

In [13]:
# 모델 컴파일              
imdb_model.get_classification_model().compile(loss="binary_crossentropy", 
                                              optimizer='rmsprop',
                                              metrics=["accuracy"])

# callback (1) - 체크포인트
checkpointer = ModelCheckpoint(filepath=train_params.model_file_path,
                                verbose=1,
                                save_best_only=True,
                                save_weights_only=True)

# callback (2) - 조기종료
early_stop = EarlyStopping(patience=2)

# 학습 시작
imdb_model.get_classification_model().fit(x_train, 
                                          y_train, 
                                          batch_size=train_params.batch_size,
                                          epochs=train_params.num_epochs,
                                          verbose=2,
                                          validation_split=0.01,
                                          callbacks=[checkpointer])

# 모델 저장
imdb_model._save_model(train_params.model_hyper_parameters)
train_params.save()

Instructions for updating:
Use tf.cast instead.
Train on 24750 samples, validate on 250 samples
Epoch 1/30
 - 40s - loss: 1.0038 - acc: 0.8686 - val_loss: 0.6622 - val_acc: 0.8560

Epoch 00001: val_loss improved from inf to 0.66215, saving model to ./checkpoint/imdb/full_model_10.hdf5
Epoch 2/30
 - 39s - loss: 0.4992 - acc: 0.8782 - val_loss: 0.4534 - val_acc: 0.8600

Epoch 00002: val_loss improved from 0.66215 to 0.45337, saving model to ./checkpoint/imdb/full_model_10.hdf5
Epoch 3/30
 - 39s - loss: 0.3740 - acc: 0.8825 - val_loss: 0.3998 - val_acc: 0.8600

Epoch 00003: val_loss improved from 0.45337 to 0.39985, saving model to ./checkpoint/imdb/full_model_10.hdf5
Epoch 4/30
 - 39s - loss: 0.3271 - acc: 0.8903 - val_loss: 0.3741 - val_acc: 0.8760

Epoch 00004: val_loss improved from 0.39985 to 0.37414, saving model to ./checkpoint/imdb/full_model_10.hdf5
Epoch 5/30
 - 39s - loss: 0.3025 - acc: 0.8960 - val_loss: 0.3723 - val_acc: 0.8680

Epoch 00005: val_loss improved from 0.37414 to 

In [14]:
# 모델 평가
imdb_model.get_classification_model().evaluate(x_test, 
                                               y_test, 
                                               batch_size=train_params.batch_size*10,
                                               verbose=2)

[0.30160063052177427, 0.8941999991416931]